# Multi-Channel EGG Processing

`egg_process_multichannel` processes a `(n_channels, n_samples)` EGG
array using one of three named strategies:

| `method` | Description |
|---|---|
| `"per_channel"` | Run `egg_process` independently on every electrode; return per-channel metrics + best-channel index |
| `"best_channel"` | Select the electrode with the highest normogastric band power, then process that channel only |
| `"ica"` | Spatially denoise all channels with FastICA, then run `per_channel` on the reconstructed data |

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import gastropy as gp

plt.rcParams["figure.dpi"] = 100
plt.rcParams["figure.facecolor"] = "white"

# Load the bundled 7-channel EGG dataset (Wolpert et al., 2020)
egg = gp.load_egg()
signal = egg["signal"]   # (7, n_samples)
sfreq = egg["sfreq"]
ch_names = list(egg["ch_names"])

print(f"{signal.shape[0]} channels, {signal.shape[1]} samples ({signal.shape[1]/sfreq:.0f} s @ {sfreq} Hz)")

## Per-Channel Processing

Process each electrode independently. Returns a summary DataFrame with
peak frequency, instability coefficient, and proportion normogastric
for every channel, plus the index of the channel with the highest
normogastric band power.

In [ ]:
result_per = gp.egg_process_multichannel(signal, sfreq, method="per_channel")

print("Per-channel summary:")
summary = result_per["summary"].copy()
summary.insert(1, "channel_name", ch_names)
print(summary.to_string(index=False, float_format="{:.4f}".format))
print(f"\nBest channel: {ch_names[result_per['best_idx']]} (index {result_per['best_idx']})")

## Best-Channel Processing

`method="best_channel"` selects the channel with the highest
normogastric band power and runs the full `egg_process` pipeline on it.
The returned dictionary has the same structure as a direct `egg_process`
call, with `info["best_channel_idx"]` added.

In [ ]:
result_best = gp.egg_process_multichannel(signal, sfreq, method="best_channel")

best_idx = result_best["info"]["best_channel_idx"]
print(f"Best channel selected: {ch_names[best_idx]} (index {best_idx})")
print(f"Peak frequency       : {result_best['info']['peak_freq_hz']:.4f} Hz "
      f"({result_best['info']['peak_freq_hz'] * 60:.2f} cpm)")
print(f"Proportion normo     : {result_best['info']['proportion_normogastric']:.0%}")
print(f"Instability coeff.   : {result_best['info']['instability_coefficient']:.4f}")

## ICA Spatial Denoising

`method="ica"` applies FastICA across all channels to find statistically
independent components. Components whose gastric-band SNR falls below
`ica_snr_threshold` are zeroed out before reconstructing the signal.
The denoised data is then processed per-channel.

> **Note:** For short or noisy recordings, lower `ica_snr_threshold`
> (e.g. `1.5`) to retain more components.

In [ ]:
result_ica = gp.egg_process_multichannel(
    signal, sfreq, method="ica",
    ica_snr_threshold=1.5,
    ica_random_state=42,
)

ica_info = result_ica["ica_info"]
print(f"ICA components: {ica_info['n_components']} total, "
      f"{ica_info['n_kept']} kept, {ica_info['n_removed']} removed")
print(f"Component SNRs: {ica_info['component_snr'].round(2)}")

print("\nPer-channel summary after ICA denoising:")
ica_summary = result_ica["summary"].copy()
ica_summary.insert(1, "channel_name", ch_names)
print(ica_summary.to_string(index=False, float_format="{:.4f}".format))

## Compare Peak Frequencies Across Methods

In [ ]:
fig, ax = plt.subplots(figsize=(9, 4))

x = np.arange(len(ch_names))
w = 0.35

peak_per = result_per["summary"]["peak_freq_hz"].values * 60
peak_ica = result_ica["summary"]["peak_freq_hz"].values * 60

ax.bar(x - w/2, peak_per, width=w, label="per_channel", color="#1f77b4", alpha=0.8)
ax.bar(x + w/2, peak_ica, width=w, label="ica", color="#ff7f0e", alpha=0.8)
ax.axhspan(2, 4, alpha=0.12, color="green", label="Normogastric (2–4 cpm)")
ax.set_xticks(x)
ax.set_xticklabels(ch_names)
ax.set_xlabel("Channel")
ax.set_ylabel("Peak frequency (cpm)")
ax.set_title("Peak Gastric Frequency per Channel: per_channel vs. ica")
ax.legend()
plt.tight_layout()
plt.show()

**See also:**
[Channel Selection](channel_selection.ipynb),
[One-Call EGG Pipeline](egg_process.ipynb),
[Artifact Removal Strategies](artifact_removal.ipynb)